# Example 10 Backreaction

One of $\texttt{DarkHistory}$'s signature features is its ability to include the effects of backreaction on the thermal and ionization histories of the universe.  In previous treatments, one assumed that the effects of dark matter energy injection were small enough that one could replace the hydrogen ionization level $x_\text{HII}$ and the IGM temperature $T_m$ by their baseline values when computed in the absence of dark matter energy injection $x_{\text{HII},0}$ and $T_{m,0}$.  On the other hand, $\texttt{DarkHistory}$ can be used without this approximation.  We will see in this notebook that there is a wide range of models for which this improved treatment is necessary.

## Import Modules

In [ ]:
%reload_ext autoreload
%autoreload 2
import sys
sys.path.append("..")

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc_file('matplotlibrc')

from scipy.interpolate import interp1d
from scipy.interpolate import interp2d
import numpy as np

import darkhistory.physics as phys
from darkhistory.history.tla import get_history

import darkhistory.main as main
import darkhistory.config as config
from tqdm import tqdm_notebook as tqdm

## Load the maximum allowed $\langle \sigma v \rangle$ and minimum $\tau$

To assess the effects of backreaction, we will compute $T_m(z)$ with and without backreaction for the cases of dark matter decaying or annihilating into $\gamma \gamma$ or $e^+ e^-$, and we will sweep over a wide range of dark matter masses, $m_\chi$.  The effects of backreaction are amplified as the lifetime $\tau$ is decreased or the velocity-averaged cross-section $\langle \sigma v \rangle$ is increased.  We will therefore use the maximum $\langle \sigma v \rangle$ and minimum $\tau$ allowed by current CMB constraints [[1]](#cite_Tracy2015), [[2]](#cite_Chih_Liang), [[3]](#cite_PLANCK).  We download these maximal values as a function of secondary particle injection energy $E_\text{inj}$.  The CMB limit data has been included in the downloaded data from [*here*](https://doi.org/10.7910/DVN/DUOUWA).

Note: It is important when downloading the 'CMB_limits*' files that the 'Original Format' option is chosen to ensure that the two .csv files containing the decay limits are downloaded correctly.

In [ ]:
f_elec_CMB_raw = np.loadtxt(config.data_path+'/CMB_limits_elec_swave.txt', delimiter=',')
log10eng_elec_CMB  = f_elec_CMB_raw[0:2760:69, 0]
log10rs_elec_CMB = f_elec_CMB_raw[0:69, 1]

f_phot_CMB_raw = np.loadtxt(config.data_path+'/CMB_limits_phot_swave.txt', delimiter=',')
log10eng_phot_CMB  = f_phot_CMB_raw[0:2800:70, 0]
log10rs_phot_CMB = f_phot_CMB_raw[0:70, 1]

f_elec_CMB_raw = np.transpose(np.reshape(f_elec_CMB_raw[:,2], (40,69)))
f_phot_CMB_raw = np.transpose(np.reshape(f_phot_CMB_raw[:,2], (40,70)))

f_elec_CMB = interp2d(log10eng_elec_CMB, log10rs_elec_CMB, f_elec_CMB_raw)
f_phot_CMB = interp2d(log10eng_phot_CMB, log10rs_phot_CMB, f_phot_CMB_raw)

decay_elec_CMB_raw = np.loadtxt(config.data_path+'/CMB_limits_elec_decay.txt', delimiter=',')
decay_phot_CMB_raw = np.loadtxt(config.data_path+'/CMB_limits_phot_decay.txt', delimiter=',')

decay_elec_CMB = interp1d(np.transpose(decay_elec_CMB_raw)[0,:], np.transpose(decay_elec_CMB_raw)[1,:])
decay_phot_CMB = interp1d(np.transpose(decay_phot_CMB_raw)[0,:], np.transpose(decay_phot_CMB_raw)[1,:])

#Derived from Planck 2018 cosmological parameters
p_ann = 3.5e-28

def xsec_bound_elec_CMB(mDM, DM_process):
    if DM_process == 'swave':
        return p_ann*(mDM*1e-9)/f_elec_CMB(np.log10(mDM-phys.me), np.log10(601))[0]
    elif DM_process == 'decay':
        return np.array([decay_elec_CMB(mDM*1e-9)])[0]

def xsec_bound_phot_CMB(mDM, DM_process):
    if DM_process == 'swave':
        return p_ann*(mDM*1e-9)/f_phot_CMB(np.log10(mDM), np.log10(601))[0]
    elif DM_process == 'decay':
        return np.array([decay_phot_CMB(mDM*1e-9)])[0]

## Comparison to Fig. 15 of [[4]](#cite_Hongwan)

To get a feel for the calculation and check $\texttt{DarkHistory}$ we will try to reproduce Fig 15. of [[4]](#cite_Hongwan).  First we establish the baseline (black-dashed curve) without dark matter energy injection.  This can be done by passing a redshift vector to [*tla.get_history()*](https://darkhistory.readthedocs.io/en/master/_autosummary/darkhistory/history/tla/darkhistory.history.tla.get_history.html).

In [ ]:
rs_vec = 3000*np.exp(-.004*np.arange(1595))
        
baseline = get_history(rs_vec)

Now we calculate $T_m(z)$ assuming no backreaction. To do so, we will use tabulated values for $f_c(z)$ and use them as inputs for [*tla.get_history()*](https://darkhistory.readthedocs.io/en/master/_autosummary/darkhistory/history/tla/darkhistory.history.tla.get_history.html).  We can do this by inputting `baseline_f = True`.

In [ ]:
no_br = get_history(
    rs_vec, baseline_f = True,
    inj_particle = 'elec', DM_process = 'decay',
    mDM=100e6, lifetime = 3e25,
)

Finally, we calculate $T_m(z)$ including backreaction by using [*main.evolve()*](https://darkhistory.readthedocs.io/en/master/_autosummary/main/main.evolve.html).  This will take a while the first time it is run because we must download the transfer functions.

In [ ]:
br_data = main.evolve(
    primary='elec_delta',
    DM_process='decay', mDM=100e6, lifetime=3e25,
    start_rs = 3000,
    coarsen_factor=16, backreaction=True
)

T_br = br_data['Tm']/phys.kB

Finally, we re-make Fig 15.

In [ ]:
fig_BR_example = plt.figure(figsize=(15,6.5))

ax = plt.subplot(1,2,1)

ax = plt.gca()
ax.loglog()

plt.xlabel('Redshift $(1+z)$')
plt.ylabel('Matter Temperature $T_m$ [K]')

plt.title(r'\bf{Backreaction Comparison}')

plt.axis([5, 3e3, 1, 1e5])

plot_std, = plt.plot(rs_vec, baseline[:,0]/phys.kB, 'k--', label='Baseline')
plot_no_BR, = plt.plot(rs_vec, no_br[:,0]/phys.kB, label='Without Backreaction', color='C0')
plot_BR, = plt.plot(br_data['rs'], br_data['Tm']/phys.kB, label='With Backreaction', color='C1')

plt.text(0.06, 0.9, r'$\chi \to e^+e^-$, $m_\chi$ = 100 MeV', transform=ax.transAxes, fontsize=20)
plt.text(0.06, 0.84, r'$\tau = 3 \times 10^{25}$ s', transform=ax.transAxes, fontsize=20)

legend = plt.legend(
    handles=[plot_no_BR, plot_BR, plot_std], loc='lower right'
)

ax = plt.subplot(1,2,2)

ax = plt.gca()
ax.loglog()

plt.xlabel('Redshift $(1+z)$')
plt.ylabel('Hydrogen Ionization Fraction $n_\mathrm{HII}/n_\mathrm{H}$')

plt.title(r'\bf{Backreaction Comparison}')

plt.axis([5, 3e3, 7e-6, 2])

plot_std, = plt.plot(rs_vec, baseline[:,1], 'k--', label='Baseline')
plot_no_BR, = plt.plot(rs_vec, no_br[:,1], label='Without Backreaction', color='C0')
plot_BR, = plt.plot(br_data['rs'], br_data['x'][:,0], label='With Backreaction', color='C1')

plt.text(0.06, 0.9, r'$\chi \to e^+e^-$, $m_\chi$ = 100 MeV', transform=ax.transAxes, fontsize=20)
plt.text(0.06, 0.84, r'$\tau = 3 \times 10^{25}$ s', transform=ax.transAxes, fontsize=20)

legend = plt.legend(
    handles=[plot_no_BR, plot_BR, plot_std], loc='lower right'
)

## Calculate With and Without back-reaction

Now we perform the same calculation, except now over a grid of energy injections.  We first specify these energies, then we define the function `get_T()` to easily calculate $T_m$ for a given mass and injection model with and without backreaction.

In [ ]:
log10_Einj_arr = np.arange(4.01, 12.51, 0.25)

When computing s-wave annihilation models, we must remember that structure formation now boosts the annihilation rate at late times. We can load one of DarkHistory's default structure formation boost factors that can be obtained using the function [*physics.struct_boost_func()*](https://darkhistory.readthedocs.io/en/master/_autosummary/darkhistory/physics/darkhistory.physics.struct_boost_func.html). The first thing we need to do is obtain the function that returns the boost: 

In [ ]:
struct_boost_einasto_subs = phys.struct_boost_func(model='einasto_subs')

In [ ]:
%autoreload

def get_T(mDM, DM_process, br, pri):
    """Gets the temperature history, decays.
    
    Parameters
    ----------
    mDM : float
        The mass of the dark matter in eV. 
    tau : float
        The decay lifetime in s. 
    br : bool
        Whether to use backreaction or not. 
    pri : {'elec', 'phot'}
        The primary particles that DM decays to. 
        
    Returns
    --------
    float
        The temperature history in K. 
    
    """
    
    if pri == 'elec':
        pri_str = 'elec_delta'
        cf = 12
        param = xsec_bound_elec_CMB(mDM, DM_process)
        rs_vec = 3000*np.exp(-.012*np.arange(552))
    if pri == 'phot':
        pri_str = 'phot_delta'
        cf = 4
        param = xsec_bound_phot_CMB(mDM, DM_process)
        rs_vec = 3000*np.exp(-.004*np.arange(1655))
        
    if DM_process == 'swave':
        struct_boost = phys.struct_boost_func(model='einasto_subs')
    else:
        struct_boost = None
        
    if br:
        result = main.evolve(
            primary=pri_str,
            DM_process=DM_process, mDM=mDM, lifetime=param, sigmav = param,
            start_rs = 3000, end_rs=4.004,
            coarsen_factor=cf, backreaction=False,
            struct_boost = struct_boost
        )
        
        return result['Tm']/phys.kB
    
    else:

        if pri == 'phot' and DM_process == 'swave':
            # Use main.evolve() here because the differential is so small that
            # using f_std leads to undesirable interpolation errors.
            result = main.evolve(
                primary=pri_str,
                DM_process=DM_process, mDM=mDM, lifetime=param, sigmav = param,
                start_rs = 3000, end_rs=4.004,
                coarsen_factor=cf, backreaction=False,
                struct_boost = struct_boost
            )
            
            return result['Tm']/phys.kB

        else:
            result = get_history(
                rs_vec, baseline_f = True,
                inj_particle = pri, DM_process = DM_process,
                mDM=mDM, lifetime = param, sigmav = param
            )
            
            return result[:,0]/phys.kB

We now loop over all injection energies and calculate $T_m$ for decay or annihilation into photons or $e^+e^-$.

In [ ]:
Tm_no_br = np.array([None for m in log10_Einj_arr])
Tm_br = np.zeros_like(Tm_no_br)

In [ ]:
%autoreload
for i, log10Einj in enumerate(tqdm(log10_Einj_arr)):
    
    Einj = 10**log10Einj
    print('****** log10(mDM): ', log10Einj, ' ******')

    Tm_no_br[i] = {
        'phot_decay': get_T(2*Einj, 'decay', br=False, pri='phot'),
        # We include this little fudge factor here because python's ode solver will sometimes guess negative 
        # values for the redshift at certain values of Einj
        'phot_swave': get_T(Einj+1e-4, 'swave', br=False, pri='phot'),
        'elec_swave': get_T(Einj+phys.me, 'swave', br=False, pri='elec'),
        'elec_decay': get_T(2*(Einj+phys.me), 'decay', br=False, pri='elec')
    }

print('###### Calculation Complete! ######')

And then we calculate with back-reaction

In [ ]:
%autoreload
for i, log10Einj in enumerate(tqdm(log10_Einj_arr)):
    
    Einj = 10**log10Einj
    print('****** log10(mDM): ', log10Einj, ' ******')
    
    Tm_br[i] = {
        'phot_decay': get_T(2*Einj, 'decay', br=True, pri='phot'),
        'phot_swave': get_T(Einj, 'swave', br=True, pri='phot'),
        'elec_swave': get_T(Einj+phys.me, 'swave', br=True, pri='elec'),
        'elec_decay': get_T(2*(Einj+phys.me), 'decay', br=True, pri='elec')
    }

print('###### Calculation Complete! ######')

## Heat Plots of the Effects of Backreaction

Now we save the data we just generated or load data what we've previously generated.  `direc` should be set to the directory where the data was (or will be) saved, e.g. `direc=/directory/where/the/data/gets/saved/`.

In [ ]:
import pickle
save_data = False
direc = '/foo/bar/'

if save_data:
    pickle.dump(Tm_no_br, open(direc+'Tm_no_br.p','wb'))
    pickle.dump(Tm_br, open(direc+'Tm_br.p','wb'))
else:
    Tm_no_br = pickle.load(open(direc+'Tm_no_br.p','rb'))
    Tm_br = pickle.load(open(direc+'Tm_br.p','rb'))

And then we plot our data

In [ ]:
inj_part = 'phot'
inj_type = 'swave'

log10_Einj_arr = np.arange(4.01, 12.51, 0.25)
log10_Einj_elec_decay_arr = np.arange(4.01, 12.51, 0.25)
log10_E = log10_Einj_arr

diff_list = np.array([Tm_br0[k][inj_part+'_'+inj_type]/Tm_no_br0[k][inj_part+'_'+inj_type]-1. for k in np.arange(34)])
diff_list[diff_list<=0]=.001
        
if inj_part == 'phot':
    cf=4
    if inj_type == 'decay':
        conv_fac = 2
        conv_add = 0
    else:
        conv_fac = 1
        conv_add = 0
else:
    if inj_type == 'swave':
        conv_fac = 1
        conv_add = phys.me
    else:
        log10_E = log10_Einj_elec_decay_arr
        conv_fac = 2
        conv_add = phys.me
    cf=12

rs_list = np.exp(np.arange(np.log(3000), np.log(4.004), step=-0.001*cf))

plt.figure(figsize=(7.7, 6.2))

ax = plt.gca()
ax.loglog()

# Specify the proper contour levels and label them
if inj_part == 'phot':
    if inj_type == 'decay':
        frac_change_levels = [1e-4, 5, 10, 20, 50, 100, 200]
        frac_change_labels = ['0\%', '5\%', '10\%', '20\%', '50\%', '100\%', '200\%']

    else:
        frac_change_levels = [1e-5, .1, .5, 1, 2, 5, 10, 20, 40]
        frac_change_labels = ['0\%','0.1\%', '0.5\%', '1\%', '2\%', '5\%', '10\%', '20\%', '40\%']
else:
    if inj_type == 'decay':
        frac_change_levels = [0.001, 5, 10, 50, 100, 200, 500, 900]
        frac_change_labels = ['0\%', '5\%', '10\%', '50\%', '100\%', '200\%', '500\%', '900\%']
    else:
        frac_change_levels = [0.001, 5, 10, 20, 50, 100, 200]
        frac_change_labels = ['0\%', '5\%', '10\%', '20\%', '50\%', '100\%', '200\%']
    
    
plt_heating = plt.contour(
    (10**log10_E + conv_add)*conv_fac, rs_list, 
    np.transpose(np.abs(diff_list)*100),
    levels=frac_change_levels,
    linewidths=0.5,
    colors='k'
)

cntr1 = ax.contourf((10**log10_E + conv_add)*conv_fac, 
                    rs_list, 
                    np.transpose(np.abs(diff_list)*100), 
                    frac_change_levels,
                    norm=matplotlib.colors.LogNorm(),
                    cmap=matplotlib.cm.inferno
)

cbar = plt.colorbar(
    cntr1, ax=ax, ticks=frac_change_levels,
)
cbar.set_ticklabels(frac_change_labels)
cbar.set_label(r'$T_m$ Fractional Change $\delta T_m/ T_{m,0}$', labelpad=20)

plt.axis([(10**log10_E[0] + conv_add)*conv_fac, (10**log10_E[-1] + conv_add)*conv_fac, 5.1, 201.61653821924926])

if inj_part == 'elec':
    if inj_type == 'decay':
        label1_txt = r'$\chi \to e^+e^-$'
        label2_txt = r'$\tau_{\min}$ from CMB'
        label1_dx = -0.05
        label2_dx = +0.12
    else:
        label1_txt = r'$\chi \chi \to e^+e^-$'
        label2_txt = r'$\langle \sigma v \rangle_{\max}$ from CMB'
        label1_dx = -0.1
        label2_dx = +0.02
else:
    if inj_type == 'decay':
        label1_txt = r'$\chi \to \gamma \gamma$'
        label2_txt = r'$\tau_{\min}$ from CMB'
        label1_dx = 0.
        label2_dx = +0.12
    else:
        label1_txt = r'$\chi \chi \to \gamma \gamma$'
        label2_txt = r'$\langle \sigma v \rangle_{\max}$ from CMB'
        label1_dx = -0.04
        label2_dx = +0.03

plt.text(0.77+label1_dx, 0.91, label1_txt,transform=ax.transAxes, color='w', fontsize=20)
plt.text(0.46+label2_dx, 0.83, label2_txt,transform=ax.transAxes, color='w', fontsize=20)

plt.title(r'\bf{Backreaction $T_m$ Comparison}')
plt.ylabel(r'Redshift ($1+z$)')
plt.xlabel(r'Dark Matter Mass $m_\chi$ [eV]')

## Bibliography

[1]<a id='cite_Tracy2015'></a> T. R. Slatyer "Indirect dark matter signatures in the cosmic dark ages. I. Generalizing the bound on s-wave dark matter annihilation from Planck results," arXiv:1506.03811, Phys. Rev. D 93, no. 2, 023527 (2016).
  
[2]<a id='cite_PLANCK'></a> N. Aghanim et al. (Planck), “Planck 2018 results. VI. Cosmological parameters,” (2018), arXiv:1807.06209.

[3]<a id='cite_Chih_Liang'></a> T. R. Slatyer and C. Wu, “General Constraints on Dark Matter Decay from the Cosmic Microwave Background,” arXiv:1610.06933, Phys. Rev. D95, 023010 (2017).

[4]<a id='cite_Hongwan'></a> Hongwan Liu, Tracy R. Slatyer, and Jesús Zavala, “Contributions to cosmic reionization from dark matter annihilation and decay,” Phys. Rev. D94, 063507 (2016),
arXiv:1604.02457 [astro-ph.CO].